# Cómo debuggear código en Julia

Los mensajes de error de **Julia** son bastante crípticos y en general no son fáciles de entender. Si has utilizado **Python**, vas a notar una gran diferencia y vas a extrañar lo explícitos que eran los mensajes de error en ese lenguaje. Sin embargo, un par de nociones generales de cómo funciona **Julia** y sobre todo mucha práctica, son útiles para entender qué es lo que está fallando en nuestro código.

Para empezar es útil decir que, como con cualquier lenguaje de programación, los errores en **Julia** se pueden clasificar en cuatro categorías:

1. Errores de **sintáxis** o de **compilación** : escribí algo mal y el compilador de **Julia** no me entiende. Puede ser por un *eror de ded o*, un signo de puntuación que olvidé o que utilicé un `elseif` sin antes haber llamado `if`. Mi progama no se ejecuta.

2. Errores de **ejecución** : mi programa está "bien" escrito, se compila y se comienza a ejecutar pero se detiene en algún momento porque se encuentra con una instrucción invalida. El ejemplo típico es una división entre cero, pero **en Julia esto no es un error, el progama regresa el número (especial) `Inf`**. Un ejemplo de un error de ejecución que sí se da en **Julia** es llamar una variable antes de que sea definida.

3. Errores **lógicos** : los más difíciles de corregir. Mi programa se ejecuta sin fallar aparentemente, pero el resultado que escupe es completamente ilógico. Ejemplo: `mi_raiz(16) = 109.23` (se supone que `mi_raiz` implementa la raíz cuadrada...).

4. Errores de **PyPlot** : bueno, estos no aparecen en todos los lenguaje sino que son particulares de **Julia**. Tmabién es cierto, estrictamente un error de **PyPlot** entra en alguna de las categorías de arriba. Lo que pasa es que **PyPlot** puede ser realmente molesto y va a concentrar buena parte de los errores que uno comete, así que merece una sección propia.

Vamos a discutir un par de consejos para saber cómo encontrar y corregir estos errores. 

In [3]:
a = 0
10/a #OJO: en Julia dividir entre cero no escupe un error!

Inf

In [62]:
typeof(Inf)

Float64

## 1. Errores de sintáxis

In [7]:
function mi_raiz(y)
    a = sin(y)
    a = 2a

LoadError: syntax: incomplete: "function" at In[7]:1 requires end
while loading In[7], in expression starting on line 1

Quizás es un ejemplo burdo, es evidente que lo que falta es un `end` y además esto es lo que dice el mensaje de error. Pero el mensaje dice más cosas que pueden ser interesantes: el error es de sintáxis, hay algo que está incompleto, y lo que falló fue la ejecución de la celda `In[7]`, en particular en la línea `1`. Esto último no es tan evidente, uno podría esperar que el error fuera en la última línea de la celda, pero en la línea `1` fue donde quise definir la función que que quedó incompleta.

---

In [5]:
function mi_raiz(y)
    a = sin(
end

LoadError: syntax: unexpected end
while loading In[5], in expression starting on line 3

De nuevo, el mensaje dice que el error es de sintáxis y que la instrucción `end` de la función es inesperado. Este mensaje sin embargo, tiene una complicación comparado con el ejemplo anterior: no hace referencia a la línea `2` que es donde claramente está el problema. 

Siempre que no encuentre un error en la línea de la que se queja **Julia**, es útil revisar lo que escribí justo antes. Al hacerlo aquí encuentro que la instrucción de `end` no se esperaba porque tengo un paréntesis abierto. **Julia** espera encontrar el paréntesis que cierra antes de `end`.

---

In [6]:
xs = [0:0.1:10]

for x in xs
    println(x, "x)
end

LoadError: syntax: incomplete: invalid string syntax
while loading In[6], in expression starting on line 3

El mensaje dice que el error es de sintáxis, que hay algo incompleto y que el problema es con una cadena. Si sé que es con una cadena, voy a buscar directamente el problema cerca de donde estoy usando cadenas. Nótese que el mensaje dice que el problema está cerca de la línea `3`, pero mi celda ya tiene suficiente código para que empiece a ser molesto contar a mano las líneas. En el modo de edición del notebook (celda seleccionada debe de estar en gris), puedo apretar `l` (házlo) y aparece el número de las líneas a la izquierda.

---

In [8]:
sin(mi_funcion(x,mi_raiz(length(y)))

LoadError: syntax: incomplete: premature end of input
while loading In[8], in expression starting on line 1

Un error más difícil de encontrar: hay algo incompleto pero de entrada no salta a la vista qué es lo que es. 

Cuando compongo funciones siempre tengo que cuidar que todas las parentesis, corchetes, etc, estén cerrados. Para ésto aprovecho que al editar la celda y poner el cursor del lado derecho de un paréntesis o corchete, él y su pareja se resaltan en verde. Así puede encontrar el paréntesis que no cerré.

---

## 2. Errores de ejecución

In [14]:
function mi_funcion(x,a)
    if a > 0
        z = 10
    end
    
    x*a*z
end

mi_funcion (generic function with 1 method)

La definición de arriba no escupe ningún error, sus sintáxis está bien. Pero si le doy un argumento negativo, obtengo un error:

In [15]:
mi_funcion(10,-2)

LoadError: z not defined
while loading In[15], in expression starting on line 1

El mensaje de error hace referencia a dos lugares: 
1. A la celda que acabo de correr `In[15]`, línea `1`.
2. A una celda anterior `In[14]`, línea `6`, dónde según el mensaje algo tiene que ver `mi_funcion`.

El mensaje es extraño porque en lo que acabo de correr no hay ninguna z. Reviso en orden las celdas a las que el mensaje hace referencia, empezando por la última que se utilizó y subiendo hasta la primera. 

En `In[15]`, línea `1`, no parece haber ningún error. El problema debe ser en la otra celda. Ahí sí hay una z, y me doy cuenta que la instrucción `z = 10` sólo se ejecuta si `a > 0`. Ahí está el problema, cuando `a < 0`, la función intenta regreasar `x*a*z` pero no sabe cuál es el valor de `z`.

---

In [20]:
xs = linspace(0,2pi,10)

ys = sin^2(xs)

LoadError: `^` has no method matching ^(::Function, ::Array{Float64,1})
while loading In[20], in expression starting on line 3

Un mensaje mucho más extraño. No tengo mucha idea de qué es lo que dice, algo de un método, y sé que el problema parece estar en la línea `3`. En este momento recuerdo que **Julia** se maneja con *métodos* y según lo que el mensaje dice problema podría ser que la función `^` (exponente) no tiene un método para lo que le estoy pidiendo.  

####Unas palabras sobre los métodos:

Supongamos que `a` y `b` son variables. Yo estoy acostumbrado a escribir 

`a + b`

pero en **Julia** los operadores, aún los más sencillos como `+`, son funciones, así que la sintáxis normal `mi_funcion(los,argumentos)` también sirve:

In [17]:
a = 10
b = 12

+(a,b)

22

De hecho esta es la manera "fundamental" de usar el operador, y con ésto explico lo que veo en el mensaje de error: no existe un método `^(cosa1, cosa2)` (donde `cosa1` es del tipo `Function` y `cosa2` el del tipo `Array{Float64,1}`).

Puedo intentar otra cosa. Tal vez a esta altura ya se me ocurrió que `sin^2(xs)` es la notación en matemáticas, pero que a **Julia** puede no gustarle que eleve al cuadrado entre la función (`sin`) y el argumento de la función (`xs`). Así que intento algo más "computacional":

In [18]:
ys = sin(xs)^2

LoadError: `*` has no method matching *(::Array{Float64,1}, ::Array{Float64,1})
while loading In[18], in expression starting on line 1

¡Otro error! Bueno, pero este es diferente, y ya entendí un poco el mensaje de los métodos: en este caso, **Julia** no tiene un método para multiplicar (con la sintáxis  `*(a,b)`) dos arreglos. Claro, ¡esto tiene sentido! ¿A poco en matemáticas puedo escribir $\vec{v} \,^2$?

Conclusión, lo que estoy intentando hacer es inválido y lo tengo que hacer de otro modo, por ejemplo con una comprensión de arreglo:

In [21]:
ys = [sin(x)^2 for x in xs]

10-element Array{Any,1}:
 0.0        
 0.413176   
 0.969846   
 0.75       
 0.116978   
 0.116978   
 0.75       
 0.969846   
 0.413176   
 5.99904e-32

---

In [23]:
aproxima(x,n) = x*n

aproximaciones = []
x = 10

for n in 1:10
    push!(aproximaciones, aproxima(x,n))
end

LoadError: [] cannot grow. Instead, initialize the array with "T[]", where T is the desired element type.
while loading In[23], in expression starting on line 6

Un error de lo más común para los que venimos de **Python**. Queremos usar `push!` para llenar una lista vacía, pero **Julia** necesita saber de qué tipo son los objetos que quiero guardar en la lista para hacerlo lo más eficientemente posible. Las listas vacías no las declaro sólo como `[]`, necesito decir qué tipo de elementos van a contener cuando las llene. utilizo por ejemplo, `Int[]`, o `Float64[]`.

---

Por cierto, hablando de llenar listas...

In [27]:
l = Int[]

push!(l,2.0)
push!(l,4.5)

LoadError: InexactError()
while loading In[27], in expression starting on line 4

Un mensaje completamente oscuro, quién sabe qué es un `InexactError`. Lo único que sé es que el problema está en la línea `4`. ¿Sí? O bueno, tal vez en la línea de arriba, ya vimos que a veces el error está arriba de la línea a la que se refiere **Julia**. Mejor vamos a hacerlo por partes:

In [28]:
l = Int[]

push!(l,2.0)
l

1-element Array{Int64,1}:
 2

Ok, el primer `push!` funciona. 

In [29]:
push!(l,4.5)

LoadError: InexactError()
while loading In[29], in expression starting on line 1

El segundo es el que está fallando. Ok... ¿Qué puede ser? Pienso mucho, reviso mi código y me doy cuenta `l` era originalmente una lista que iba a contener enteros, y que yo estoy tratando de meterle flotantes. Pero meterle el flotante `2.0` no produjo un error, y `4.5` sí, ¿cómo puede ser esto posible?

Lo que hace **Julia** es que como dije que la lista iba a contener enteros, trata de convertir los argumentos de `push!` a tipo entero. `2.0` es "casi" un entero, sólo me deshago del punto y me quedo con `2`, pero ¿`4.5`? ¿Lo redondeo hacia 4 o hacia 5? **Julia** no va a tomar esa decisión arbitraria por nosotros, y nos escupe este error.

Por cierto, puedo revisar la documentación de la función `convert`, que se está llamando a escondidas dentro de `push!`.

In [31]:
convert(Int,2.0)

2

In [32]:
convert(Int,4.5)

LoadError: InexactError()
while loading In[32], in expression starting on line 1

---

##3. Errores lógicos

Como podrás imaginar, estos errores son distintos en cada caso concreto, así que no hay un método general para corregirlos. Sin embargo, hay una serie de "trucos" para tratar de ubicar un error de este tipo.

In [48]:
function condicion_proba(p, num_iters)
    # Genera varios números aleatorios y cuenta cuantos son menores que p ∈ [0,1]
    
    numeros  = rand(num_iters) # Esto genera num_iters números
    # Defino un contador que no contiene nada importante en un principio
    contador = rand()
    
    for num in numeros
        # Icializo el contador, al principio vale cero
        contador = 0
        if num < p
            contador += 1
        end
    end
    
    contador
end

condicion_proba (generic function with 1 method)

La función se definió sin errores de sintáxis.

In [49]:
p = 0.5
num_iters = 10

condicion_proba(p, num_iters)

1

In [50]:
condicion_proba(p, num_iters)

0

Hay algo raro, estoy haciendo básicamente "volados" y de diez que hice, ninguno cayó en el intervalo $[0, \frac{1}{2}]$. Yo esperaría algo como del orden de $5$, pero tal vez me tocó una fluctuación. ¿Y si llamo varias veces a la función?

In [53]:
for i in 1:5
    println(condicion_proba(p, num_iters))
end

1
1
1
0
0


Definitivamente hay algo incorrecto, no es una fluctuación. En un llamado a la función, que hace diez iteraciones, a lo mucho se cuenta un número que satiface la condición `num < p`, y peor aún un par de veces no se satisface nunca. 

Voy a agregar `println`, o aún mejor, el macro `@show` (que da el nombre de la variable y su valor), en algunos lugares estratégicos de mi definición.

In [54]:
function condicion_proba(p, num_iters)
    # Genera varios números aleatorios y cuenta cuantos son menores que p ∈ [0,1]
    
    numeros  = rand(num_iters) # Esto genera num_iters números
    @show numeros
    # Defino un contador que no contiene nada importante en un principio
    contador = rand()
    @show contador
    
    for num in numeros
        # Icializo el contador, al principio vale cero
        contador = 0
        if num < p
            contador += 1
            @show contador
        end
    end
    
    contador
end

condicion_proba (generic function with 1 method)

In [57]:
p = 0.5
num_iters = 5

condicion_proba(p, num_iters)

numeros => [0.9984232116077258,0.9688789040881209,0.7452930986662594,0.4108716281618454,0.04008953082680344]
contador => 0.197904275755715
contador => 1
contador => 1


1

Veo que la lista `numeros` contiene (como debe de ser), $5$ números aleatorios y puedo contar "a mano" que los que son menores que $p = 0.5$ son $2$ (los últimos). Puedo ver que cuando inicializo `contador` no contiene nada interesante, pero después aparece dos veces con el valor $1$, que es el que finalmente se regresa. Aquí tiene que estar el problema.

Pensándolo un poco más, el `@show` que puse sólo se ejecuta cuando agarro de la lista un número que sí satisface `num < p`, así que estoy cachando contador dos veces, después de que se le suma uno. Ésto significa que las dos veces que lo caché, tenía originalmente el valor cero. ¡Ahí está el problema! Al emepezar la codición `if`, `contador` ¡siempre es igual a cero! 

Reviso mi código y encuentro el problema: la instrucción `contador = 0` se ejecuta en cada iteración del bucle. Esta línea no debería de estar dentro del `for`, sino antes de él.

In [59]:
function condicion_proba(p, num_iters)
    # Genera varios números aleatorios y cuenta cuantos son menores que p ∈ [0,1]
    
    numeros  = rand(num_iters) # Esto genera num_iters números
    # Icializo el contador, al principio vale cero
    contador = 0
    
    for num in numeros
        if num < p
            contador += 1
        end
    end
    
    contador
end

p = 0.5
num_iters = 10

condicion_proba(p, num_iters)

7

---

El siguiente ejemplo no es un error, pero se puede convertir en uno cuando utilizamos el código que sigue, creyendo que hacemos otra cosa: 

In [60]:
# Los datos originale
a = [1:3]

# Guardo una copia de los datos
b = a

# Y luego modifico los datos originales
a[1] = 19
a[3] = -2

# Qué sucedió con la copia?
b

3-element Array{Int64,1}:
 19
  2
 -2

¡La copia cambió, a pesar de que sólo modificamos a `a`! **1Cuidado!** Lo que sucede es que la línea `b = a` no está duplicando lo que `a` tiene guardado, más bien dice "haz que el símbolo `b` apunte al lugar donde está guardado `a`". Cuando `a`cambia, `b` sigue apuntando al mismo lugar. 

Suena completamente ilógico, pero uno de las principales motivaciones en el diseño de **Julia** es que sea lo más rápido posible, y que pueda competir con `C` o `Fortran`. Para que así sea, hay una buena razón para que funcione de esta manera. Hacer que `b` "apunte" a la memoria donde está guardado `a` es mucho más rápido que copiar `a` a un nuevo lugar en la memoria y guardarlo por duplicado. Imagina que `a` tuviera cientos de miles o millones de elementos...

Puedo darle la vuelta a este comportamiento, usando la función `copy`.

In [61]:
# Los datos originale
a = [1:3]

# Guardo una copia de los datos
b = copy(a)

# Y luego modifico los datos originales
a[1] = 19
a[3] = -2

# Qué sucedió con la copia?
b

3-element Array{Int64,1}:
 1
 2
 3

##4. Errores de PyPlot

Hay toneladas de errores relacionados con `PyPlot`... A continuación unos pocos.

In [63]:
using PyPlot

INFO: Loading help data...


In [65]:
# Datos
xs = [1:10]

# Inicializo una lista
ys = [1]

# Y la lleno haciéndolo algo a los datos originales
for x in xs
    push!(ys, x^2)
end

In [66]:
plot(xs,ys)

LoadError: PyError (:PyObject_Call) <type 'exceptions.ValueError'>
ValueError(u'x and y must have same first dimension',)
  File "/Applications/anaconda/lib/python2.7/site-packages/matplotlib/pyplot.py", line 3099, in plot
    ret = ax.plot(*args, **kwargs)
  File "/Applications/anaconda/lib/python2.7/site-packages/matplotlib/axes/_axes.py", line 1373, in plot
    for line in self._get_lines(*args, **kwargs):
  File "/Applications/anaconda/lib/python2.7/site-packages/matplotlib/axes/_base.py", line 304, in _grab_next_args
    for seg in self._plot_args(remaining, kwargs):
  File "/Applications/anaconda/lib/python2.7/site-packages/matplotlib/axes/_base.py", line 282, in _plot_args
    x, y = self._xy_from_xy(x, y)
  File "/Applications/anaconda/lib/python2.7/site-packages/matplotlib/axes/_base.py", line 223, in _xy_from_xy
    raise ValueError("x and y must have same first dimension")

while loading In[66], in expression starting on line 1

Obtenemos un mensaje de error enorme, con muchas líneas incomprensible. Sin embargo, hay una línea interesante, justo arriba de `while loading In[66]...`:

`raise ValueError("x and y must have same first dimension")`.

Poniendo un poco más de atención este mensaje aparece de hecho (casi idéntico) en la segunda línea del error: `ValueError(u'x and y must have same first dimension',)`.

Cuando manejamos `PyPlot` y obtenemos un error, las líneas que en general me van a decir algo son esas dos, la segunda y la que está justo antes de `while loading...`.

En este caso, una vez que encontré el único pedazo de información útil en el gran mensaje de error, es bastante claro cuál es el problema: `xs` y `ys` no tienen el mismo número de elementos.

In [68]:
length(xs) , length(ys)

(10,11)

---

In [72]:
plot(rand(10), rand(10), 'r')

LoadError: PyError (:PyObject_Call) <type 'exceptions.ValueError'>
ValueError(u'third arg must be a format string',)
  File "/Applications/anaconda/lib/python2.7/site-packages/matplotlib/pyplot.py", line 3099, in plot
    ret = ax.plot(*args, **kwargs)
  File "/Applications/anaconda/lib/python2.7/site-packages/matplotlib/axes/_axes.py", line 1373, in plot
    for line in self._get_lines(*args, **kwargs):
  File "/Applications/anaconda/lib/python2.7/site-packages/matplotlib/axes/_base.py", line 304, in _grab_next_args
    for seg in self._plot_args(remaining, kwargs):
  File "/Applications/anaconda/lib/python2.7/site-packages/matplotlib/axes/_base.py", line 266, in _plot_args
    raise ValueError('third arg must be a format string')

while loading In[72], in expression starting on line 1

Usando lo que ya aprendí, me fijo en la segunda línea y encuentro:

`'third arg must be a format string'`

Hay un problema con el tercer argumento, y si nos queda duda de cuál es el tercer argumento, hay un problema con una cadena.

Ya habíamos visto en clase que `'r'` es un caracter (`Char`), y `"r"` una cadena (`ASCIIString`). Resulta que lo que yo tengo que pasar como argumento es una cadena, con comillas dobles. Esto es confuso porque la documentación `?plot` habla de cosas con comillas simples, pero esa es la documentación de **matplotlib** en **Python** donde no hay diferencia entre los dos tipos de comillas.

---

In [76]:
muestra = rand(10_000)
plt.hist(muestra, normed = True)

LoadError: True not defined
while loading In[76], in expression starting on line 2

Revisando la documentación de la función histograma `plt.hist`, si quiero normalizar la gráfica uso `normed = True`. De nuevo, esto es en **Python**, en **Julia** la variable booleana es `true`con minúscula.

---